In [1]:
import os
os.remove('snorkel.db') #need these two files to exist in sandbox directory. Just copy from tutorials
os.system('cp snorkel.db\ corpus snorkel.db');

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession
session = SnorkelSession()

import sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

In [3]:
from snorkel.models import Corpus

corpus = session.query(Corpus).filter(Corpus.name == 'Hardware').one()
print "%s contains %d Documents" % (corpus, len(corpus))

Corpus (Hardware) contains 97 Documents


In [4]:
# Make parts list
from hardware_utils import get_gold_dict
from collections import defaultdict
gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_dev/hardware_dev_gold.csv'
gold_parts = get_gold_dict(gold_file, doc_on=True, part_on=True, val_on=False)
parts_by_doc = defaultdict(set)
for part in gold_parts:
    parts_by_doc[part[0]].add(part[1])

In [5]:
from hardware_utils import OmniNgramsPart, load_extended_parts_dict, part_rgx
from snorkel.models import candidate_subclass
from snorkel.matchers import RegexMatchSpan

Part = candidate_subclass('Part', ['part'])
part_ngrams = OmniNgramsPart(n_max=3, parts_by_doc=None)
part_matcher = RegexMatchSpan(rgx=part_rgx())

In [6]:
from hardware_utils import get_gold_dict

gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_dev/hardware_dev_gold.csv'
gold_parts = get_gold_dict(gold_file, doc_on=True, part_on=True, val_on=False)
print "%d gold parts loaded" % len(gold_parts)

809 gold parts loaded


In [7]:
from snorkel.candidates import CandidateExtractor

ce = CandidateExtractor(Part, 
                        [part_ngrams], 
                        [part_matcher],
                        throttler=None)

In [8]:
%time candidates = ce.extract(corpus.documents, 'Hardware Candidates', session)
print "%s contains %d Candidates" % (candidates, len(candidates))

[========================================] 100%
CPU times: user 40.6 s, sys: 924 ms, total: 41.5 s
Wall time: 44.1 s
Candidate Set (Hardware Candidates) contains 6976 Candidates


In [9]:
from hardware_utils import parts_f1

%time tp, fp, fn = parts_f1(candidates, gold_parts)

Scoring on Entity-Level Gold Data
Corpus Precision 0.324
Corpus Recall    0.89
Corpus F1        0.475
----------------------------------------
TP: 720 | FP: 1502 | FN: 89

CPU times: user 24.8 s, sys: 493 ms, total: 25.3 s
Wall time: 25.4 s


In [10]:
from pprint import pprint
pprint(sorted(fn)[:100])

[('BC337', 'BC337-16'),
 ('BC337', 'BC337-25'),
 ('BC337', 'BC337-40'),
 ('BC337', 'BC338-25'),
 ('BC550', 'BC546A'),
 ('BC550', 'BC546B'),
 ('BC550', 'BC546C'),
 ('BC550', 'BC547A'),
 ('BC550', 'BC547B'),
 ('BC550', 'BC547C'),
 ('BC550', 'BC548A'),
 ('BC550', 'BC548B'),
 ('BC550', 'BC548C'),
 ('BC550', 'BC549A'),
 ('BC550', 'BC549B'),
 ('BC550', 'BC549C'),
 ('BC550', 'BC550A'),
 ('BC550', 'BC550B'),
 ('BC550', 'BC550C'),
 ('BC818', 'BC817-16'),
 ('BC818', 'BC817-25'),
 ('BC818', 'BC817-40'),
 ('BC818', 'BC818-16'),
 ('BC818', 'BC818-25'),
 ('BC818', 'BC818-40'),
 ('FAIRS19194-1', 'BC856A'),
 ('FAIRS19194-1', 'BC856B'),
 ('FAIRS19194-1', 'BC856C'),
 ('FAIRS19194-1', 'BC857A'),
 ('FAIRS19194-1', 'BC857B'),
 ('FAIRS19194-1', 'BC857C'),
 ('FAIRS19194-1', 'BC858A'),
 ('FAIRS19194-1', 'BC858B'),
 ('FAIRS19194-1', 'BC858C'),
 ('FAIRS19194-1', 'BC859A'),
 ('FAIRS19194-1', 'BC859B'),
 ('FAIRS19194-1', 'BC859C'),
 ('FAIRS19194-1', 'BC860A'),
 ('FAIRS19194-1', 'BC860B'),
 ('FAIRS19194-1', 'BC860

The End.